# Concelhos list

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random

distrito_links = [
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/aveiro",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/beja",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/braga",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/braganca",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/castelo-branco",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/coimbra",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/evora",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/faro",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/guarda",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-da-graciosa",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-da-madeira",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-das-flores",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-de-porto-santo",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-de-santa-maria",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-de-sao-jorge",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-de-sao-miguel",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-do-corvo",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-do-faial",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-do-pico",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/ilha-terceira",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/leiria",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/lisboa",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/portalegre",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/porto",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/santarem",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/setubal",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/viana-do-castelo",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/vila-real",
    "https://www.imovirtual.com/pt/resultados/comprar/apartamento/viseu"
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def get_concelhos(dist_url):
    print(f"Scraping Concelhos from: {dist_url}")
    try:
        response = requests.get(dist_url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Get the district slug (e.g., 'lisboa')
        dist_slug = dist_url.split('/')[-1]
        concelho_links = []
        
        # Find all links on the page
        for a in soup.find_all('a', href=True):
            href = a['href']
            # Imovirtual links for concelhos look like: .../apartamento/lisboa/cascais
            if f"/apartamento/{dist_slug}/" in href:
                # Count segments to ensure it's a Concelho (1 segment after distrito)
                path_after_dist = href.split(f"/{dist_slug}/")[-1].strip('/')
                if "/" not in path_after_dist and path_after_dist != "":
                    concelho_links.append(href)
        
        return list(set(concelho_links)) # Remove duplicates
    except Exception as e:
        print(f"Error: {e}")
        return []

all_concelhos = {}

for url in distrito_links:
    dist_name = url.split('/')[-1]
    links = get_concelhos(url)
    all_concelhos[dist_name] = links
    print(f"  Found {len(links)} concelhos.")
    time.sleep(random.uniform(1, 2)) # Avoid being blocked

# Save results
with open('concelhos.json', 'w', encoding='utf-8') as f:
    json.dump(all_concelhos, f, indent=4, ensure_ascii=False)

print("\nConcelho links saved to concelhos.json")

# Freguesias list

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random

BASE_URL = "https://www.imovirtual.com"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def get_freguesia_links():
    try:
        with open('concelhos.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print("Error: 'concelhos.json' not found.")
        return

    freguesia_links_total = []

    # Iterate through each Distrito and its respective Concelhos
    for distrito, concelhos in data.items():
        print(f"\nProcessing District: {distrito.upper()}")
        
        for c_path in concelhos:
            # Skip possible "arrendar" links
            if "/arrendar/" in c_path:
                continue
                
            full_url = BASE_URL + c_path
            print(f"Searching freguesias in: {c_path}")
            
            try:
                response = requests.get(full_url, headers=HEADERS, timeout=15)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # O slug do concelho é a última parte do path (ex: 'sintra')
                c_slug = c_path.split('/')[-1]
                
                count_freg = 0
                for a in soup.find_all('a', href=True):
                    href = a['href']
                    
                    # Filter's logic:
                    # 1. Must contain the concelho path
                    # 2. Must be longer than the concelho path
                    # 3. Can't be an individual ad (.html)
                    if c_path in href and len(href) > len(c_path) and not href.endswith('.html'):
                        
                        # Garantees it's a full URL
                        full_freg_link = href if href.startswith('http') else BASE_URL + href
                        
                        # Final format validation:apartamento/distrito/concelho/freguesia
                        segments = full_freg_link.split('/apartamento/')[-1].strip('/').split('/')
                        
                        if len(segments) == 3:
                            freguesia_links_total.append(full_freg_link)
                            count_freg += 1
                
                print(f"    -> Found {count_freg} freguesias.")
                
            except Exception as e:
                print(f"    Error processing {c_path}: {e}")
            
            time.sleep(random.uniform(1.0, 2.5)) # Avoid being blocked

    # Save the final result
    final_list = list(set(freguesia_links_total)) # Remove duplicates
    with open('freguesias_lista.txt', 'w', encoding='utf-8') as f:
        for link in sorted(final_list):
            f.write(link + '\n')

    print(f"\nDone. {len(final_list)} links saved in 'freguesias_list.txt'.")

if __name__ == "__main__":
    get_freguesia_links()